<a href="https://colab.research.google.com/github/4en1x/diploma/blob/master/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt update -qq;
!wget https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!dpkg -i cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb;
!apt-key add /var/cuda-repo-8-0-local-ga2/7fa2af80.pub;
!apt-get update -qq;
!apt-get install gcc-4.8 g++-4.8 -y -qq;
!ln -s /usr/bin/gcc-4.8 /usr/local/cuda/bin/gcc;
!ln -s /usr/bin/g++-4.8 /usr/local/cuda/bin/g++;
!apt install cuda-8.0;
!update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-4.8 60 --slave /usr/bin/g++ g++ /usr/bin/g++-4.8

51 packages can be upgraded. Run 'apt list --upgradable' to see them.
--2019-01-17 12:26:00--  https://developer.nvidia.com/compute/cuda/8.0/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.162.216
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.162.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?4HCwlZY94kbvP25zUWsSnIJS7qKyBLW9GJgp8CufEZnULfETmWZ4kK8vAVS09waDGphq1xQuiShwVwY91d-x0xy8u0Z6R0SvnMl0bownNaxHbFfodU0Ojzhib4WaS7fmJ3EDM1zgSdaeOqIXfkMeD89oadjHyYU-jateU7TUHAQRr8jgpggoL-WIcMyxsu6F7bBT2aRk0o9nhzPfoqY-VEiB_Q [following]
--2019-01-17 12:26:00--  https://developer.download.nvidia.com/compute/cuda/8.0/secure/Prod2/local_installers/cuda-repo-ubuntu1604-8-0-local-ga2_8.0.61-1_amd64.deb?4HCwlZY94kbvP25zUWsSnI

In [2]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2016 NVIDIA Corporation
Built on Tue_Jan_10_13:22:03_CST_2017
Cuda compilation tools, release 8.0, V8.0.61


In [3]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-evm4qk2l
  Running setup.py bdist_wheel for NVCCPlugin ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-nuiex458/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [4]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [0]:
!cd /usr/src
!ln -sfn linux-headers-2.6.35-28-generic linux

CODE (AT LEAST NOW)

In [0]:
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [15]:
%%cu
#include <cuda_runtime.h>
#include "device_launch_parameters.h"
#include <stdlib.h>
#include <stdio.h>
#include "linux/kernel.h"
#include <iostream>
#include <ctime>
#include <iomanip>
#include <math.h>
#include <cmath>

#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort = true) {
	if (code != cudaSuccess) {
		fprintf(stderr, "GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
		if (abort) {
			exit(code);
		}
	}
}

#define ITERATIONS 8
#define N 4096

#define e 2.71
#define Dx 1
#define h ((float) Dx / N)
#define h2 powf(h, 2)

#define max(x, y) (((x) > (y)) ? (x) : (y))
#define min(x, y) (((x) < (y)) ? (x) : (y))

#define r1 16
#define r2 16

#define t1 2
#define t2 2

__device__
float f(int i, int j) {
	return powf(e, i * h + j * h);
}

__device__
int u(float* matrix, int i, int j) {
	return matrix[N*i + j];
}

__device__
void tile(float* matrix, int iGl, int jGl, bool isBlack) {
	int iBegin = iGl * r1 + threadIdx.x * t1;
	int jBegin = jGl * r2 + threadIdx.y * t2;

	for (size_t i = max(iBegin, 1); i < min(iBegin + t1, N - 1); i++) {
		int offset = i % 2;
		if (isBlack) {
			offset = 1 - offset;
		}
		for (size_t j = max(jBegin + offset, 1); j < min(jBegin + t2, N - 1); j+=2) {
			matrix[N*i + j] = 0.25 * (u(matrix, i + 1, j)
				+ u(matrix, i - 1, j)
				+ u(matrix, i, j + 1)
				+ u(matrix, i, j - 1) - h2 * f(i, j));
		}
	}
}

__global__
void calculateBlack(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, true);
}

__global__
void calculateRed(float* matrix) {
	tile(matrix, blockIdx.x, blockIdx.y, false);
}

float func(int i, int j) {
	return powf(e, i * h + j * h);
}

float* createRandomMatrix(int row, int col) {
	srand(static_cast <unsigned> (time(0)));
	float* matrix = new float[row * col];
	for (size_t i = 0; i < row; i++) {
		for (size_t j = 0; j < col; j++) {
			if (i == 0 || i == row - 1 || j == 0 || j == col - 1) {
				matrix[i] = func(i, j);
			} else {
				matrix[i] = 0;
			}
		}
	}
	return matrix;
}

float calculateTolerance(float* matrix1, float* matrix2) {
	float tolerance = 0;
	for (size_t i = 0; i < N * N; i++) {
		tolerance += (matrix1[i] - matrix2[i]) * (matrix1[i] - matrix2[i]);
	}
	return tolerance;
}

int main() {
	float* matrixHost = createRandomMatrix(N, N);
	float* previousMatrixHost = new float[N * N];
	float* matrixDevice;
	
	int matrixSize = sizeof(float) * N * N;

	gpuErrchk(cudaMalloc((void**)&matrixDevice, matrixSize));
	gpuErrchk(cudaMemcpy(matrixDevice, matrixHost, matrixSize, cudaMemcpyHostToDevice));

	long startTime = clock();
	dim3 blocks(N / r1, N / r2);
	dim3 threads(r1 / t1, r2 / t2);

	for (size_t i = 0; i < ITERATIONS; i++) {
		gpuErrchk(cudaMemcpy(previousMatrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost));
		calculateBlack <<<blocks, threads >>>(matrixDevice);
		gpuErrchk(cudaDeviceSynchronize());
		calculateRed <<<blocks, threads >>>(matrixDevice);
		gpuErrchk(cudaPeekAtLastError());
		gpuErrchk(cudaDeviceSynchronize());
		gpuErrchk(cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost));
		std::cout << std::setprecision(8) << std::fixed << calculateTolerance(matrixHost, previousMatrixHost) << std::endl;
	}
	std::cout << ITERATIONS << " iteration norm: " << std::setprecision(8) << std::fixed << calculateTolerance(matrixHost, previousMatrixHost) << std::endl;
	std::cout << "Calculation take time: " << clock() - startTime << std::endl;
	cudaError_t cudaResult = cudaMemcpy(matrixHost, matrixDevice, matrixSize, cudaMemcpyDeviceToHost);

	if (cudaResult != cudaSuccess) {
		std::cout << "Cuda error: " << cudaResult << std::endl;
	}
	std::system("pause");
	return 0;
}


'6838.87500000\n907.12500000\n0.75000000\n0.75000000\n0.75000000\n0.75000000\n0.75000000\n0.75000000\n8 iteration norm: 0.75000000\nCalculation take time: 860571\nsh: 1: pause: not found\n'